In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from collections import defaultdict
from copy import deepcopy
import torch

print("CUDA", torch.cuda.is_available())
import yaml
import pyhocon
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import simpletransformers
from sklearn.metrics import classification_report
from RobotCommandClassifier import utils
from RobotCommandClassifier import *
from MultilabelML import PrepareData, PrepareInput_for_simpletransformers, PrepareModel

CUDA True


In [2]:
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/CustomML.conf")
#CONFIG = configFileContent['MyMultiTiny2_data2_fold0'].as_plain_ordered_dict()

with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiTiny2_data2/fold_0/config.json", "r") as f:
    CONFIG = json.load(f)
# если хотим протестировать на всем тесте, включая фолды, на которых он обучался
#CONFIG["Data"].pop("test_only_on_fold")

In [3]:
train_x_df, train_y_df, valid_x_df, valid_y_df, test_x_df, test_y_df = PrepareData(CONFIG)

In [4]:
CONFIG["Model"]['model_name'] = CONFIG["output_dir"] + '/models/checkpoint-63780-epoch-10'
model = PrepareModel(CONFIG)

In [5]:
predictions, raw_outputs = model.predict(test_x_df.values.tolist())

  0%|          | 0/264 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
def softmax(x):
    e_x = x - np.expand_dims(np.max(x, axis=1), axis=1)
    return e_x / np.expand_dims(e_x.sum(axis=1), axis=1)
softmax_outputs = np.zeros_like(raw_outputs)
shift = 0
for num_sublabels in CONFIG['Model']['num_sublabels_per_biglabel']:
    softmax_outputs[:, shift:shift + num_sublabels] = softmax(raw_outputs[:, shift:shift + num_sublabels])
    shift += num_sublabels

In [7]:
# analyze confidence
number_of_low_confidence = 0
number_of_high_confidence = 0
for i in range(len(softmax_outputs)):
    shift = 0
    predicted_tuples = []
    for cat_i in range(len(CONFIG["Model"]['num_sublabels_per_biglabel'])):
        probs = softmax_outputs[i, shift:shift+CONFIG["Model"]['num_sublabels_per_biglabel'][cat_i]]
        assert probs.sum()>=0.9999
        chosen_probs = np.where(probs > 0.7)[0]
        if len(chosen_probs)==0:
            #chosen_probs = [np.argmax(probs)]
            #print(i, "Low confidence", CONFIG["Data"]["target_columns"][cat_i], np.round(probs, 3))
            number_of_low_confidence += 1
        else:
            #print(i, "High confidence", CONFIG["Data"]["target_columns"][cat_i], np.round(probs, 3))
            number_of_high_confidence += 1
        shift += CONFIG["Model"]['num_sublabels_per_biglabel'][cat_i]
print(number_of_low_confidence, number_of_high_confidence)

2640 528
